<a href="https://colab.research.google.com/github/Ali-Fartout/People-Clothing-Segmentation/blob/main/EDA_PCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download data

In [1]:
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d rajkumarl/people-clothing-segmentation

 99% 612M/616M [00:04<00:00, 124MB/s]
100% 616M/616M [00:04<00:00, 138MB/s]


In [2]:
import zipfile
with zipfile.ZipFile("/content/people-clothing-segmentation.zip", 'r') as zip_ref:
    zip_ref.extractall()


In [3]:
!mkdir data
!mkdir data/train data/test
!mkdir data/train/image data/train/mask data/test/image data/test/mask

In [4]:
%cd png_images/IMAGES
!find . -type f -name "img_0[0-7][0-9][0-9]\.png" -exec cp {} /content/data/train/image \;
!find . -type f -name "img_0[8-9][0-9][0-9]\.png"  -exec cp {} /content/data/test/image \;
!find . -type f -name "img_1000.png"  -exec cp {} /content/data/test/image \;

/content/png_images/IMAGES


In [5]:
%cd /content/png_masks/MASKS
!find . -type f -name "seg_0[0-7][0-9][0-9]\.png" -exec cp {} /content/data/train/mask \;
!find . -type f -name "seg_0[8-9][0-9][0-9]\.png"  -exec cp {} /content/data/test/mask \;
!find . -type f -name "seg_1000.png"  -exec cp {} /content/data/test/mask \;

/content/png_masks/MASKS


# Cleaning data

In [38]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class SegmentationDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_dir = os.path.join(self.root_dir, 'image')
        self.mask_dir = os.path.join(self.root_dir, 'mask')
        self.image_filenames = os.listdir(self.image_dir)
        self.mask_filenames = os.listdir(self.mask_dir)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # load image and mask
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_filenames[idx])
        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')

        # apply transformations, if any
        if self.transform is not None:
            for t in self.transform:
                image, mask = self.transform(image, mask)
        
        # convert PIL Image to tensor
        image = torch.tensor(np.array(image).transpose(2, 0, 1)).float()
        mask = torch.tensor(np.array(mask)).long()

        return image, mask


In [47]:

import torchvision.transforms.functional as TF


def brightness( x, y):
        x = TF.adjust_brightness(x, 0.1)
        return x, y
    
def gamma(x, y):
        x = TF.adjust_gamma(x, 0.1)
        return x, y
    
def hue( x, y):
        x = TF.adjust_hue(x, -0.1)
        return x, y
    
def crop(x, y):
        # convert to PIL images
        x = TF.to_pil_image(x)
        y = TF.to_pil_image(y)
        # perform central crop
        i, j, h, w = TF.resized_crop(x, 0, 0, min(x.size), min(x.size), size=(128, 128))
        x = TF.resized_crop(x, i, j, h, w, size=(128, 128))
        y = TF.resized_crop(y, i, j, h, w, size=(128, 128))
        # convert back to tensors
        x = TF.to_tensor(x)
        y = TF.to_tensor(y)
        # cast mask to integers
        y = y.type(torch.uint8)
        return x, y
    
def flip_hori(x, y):
        x = TF.hflip(x)
        y = TF.hflip(y)
        return x, y
    
def flip_vert(x, y):
        x = TF.vflip(x)
        y = TF.vflip(y)
        return x, y
    
def rotate(x, y):
        # convert to PIL images
        x = TF.to_pil_image(x)
        y = TF.to_pil_image(y)
        # perform rotation
        x = TF.rotate(x, 90)
        y = TF.rotate(y, 90)
        # convert back to tensors
        x = TF.to_tensor(x)
        y = TF.to_tensor(y)
        # cast mask to integers
        y = y.type(torch.uint8)
        return x, y


In [44]:
augmentations = [
  brightness,
  gamma,

]

In [48]:
from torchvision.datasets import DatasetFolder
from torchvision.transforms import transforms
import torchvision



train_dataset = DatasetFolder('/content/data/train', loader=torchvision.io.image.read_image, extensions=('png',), transform=augmentations)
val_dataset = DatasetFolder('/content/data/test', loader=torchvision.io.image.read_image, extensions=('png',))

In [50]:
len(train_dataset)

2

In [18]:
import torch.nn as nn
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)
    
    
class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.double_conv = DoubleConv(in_channels, out_channels)
        self.down_sample = nn.MaxPool2d(2)

    def forward(self, x):
        skip_out = self.double_conv(x)
        down_out = self.down_sample(skip_out)
        return (down_out, skip_out)

    
class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels, up_sample_mode):
        super(UpBlock, self).__init__()
        if up_sample_mode == 'conv_transpose':
            self.up_sample = nn.ConvTranspose2d(in_channels-out_channels, in_channels-out_channels, kernel_size=2, stride=2)        
        elif up_sample_mode == 'bilinear':
            self.up_sample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            raise ValueError("Unsupported `up_sample_mode` (can take one of `conv_transpose` or `bilinear`)")
        self.double_conv = DoubleConv(in_channels, out_channels)

    def forward(self, down_input, skip_input):
        x = self.up_sample(down_input)
        x = torch.cat([x, skip_input], dim=1)
        return self.double_conv(x)

    
class UNet(nn.Module):
    def __init__(self, out_classes=2, up_sample_mode='conv_transpose'):
        super(UNet, self).__init__()
        self.up_sample_mode = up_sample_mode
        # Downsampling Path
        self.down_conv1 = DownBlock(3, 64)
        self.down_conv2 = DownBlock(64, 128)
        self.down_conv3 = DownBlock(128, 256)
        self.down_conv4 = DownBlock(256, 512)
        # Bottleneck
        self.double_conv = DoubleConv(512, 1024)
        # Upsampling Path
        self.up_conv4 = UpBlock(512 + 1024, 512, self.up_sample_mode)
        self.up_conv3 = UpBlock(256 + 512, 256, self.up_sample_mode)
        self.up_conv2 = UpBlock(128 + 256, 128, self.up_sample_mode)
        self.up_conv1 = UpBlock(128 + 64, 64, self.up_sample_mode)
        # Final Convolution
        self.conv_last = nn.Conv2d(64, out_classes, kernel_size=1)

    def forward(self, x):
        x, skip1_out = self.down_conv1(x)
        x, skip2_out = self.down_conv2(x)
        x, skip3_out = self.down_conv3(x)
        x, skip4_out = self.down_conv4(x)
        x = self.double_conv(x)
        x = self.up_conv4(x, skip4_out)
        x = self.up_conv3(x, skip3_out)
        x = self.up_conv2(x, skip2_out)
        x = self.up_conv1(x, skip1_out)
        x = self.conv_last(x)
        return x
    




In [19]:
# Get UNet model
model = UNet()
model = model.to(device)